In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

data = pd.read_csv("data/38_feature.csv",index_col=0)
data_norm = pd.read_csv("data/38_feature_norm.csv",index_col=0)
data_std_scale = pd.read_csv("data/38_feature_std_scale.csv",index_col=0)

X = data.iloc[:,2:]
y2 = data['class2']
y4 = data['class4']
X_norm = data_norm.iloc[:,2:]
X_std_scale = data_std_scale.iloc[:,2:]


In [3]:

# define dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_norm, y2, stratify=y2 ,test_size=0.20, random_state=42)

In [25]:
from sklearn.linear_model import LogisticRegression
LR_clf =LogisticRegression(C=20,penalty='l2',solver='liblinear',random_state=1,max_iter=1000).fit(X_train, y_train)

from sklearn.metrics import accuracy_score
print('accuracy on train set',accuracy_score(LR_clf.predict(X_train),y_train))
print('accuracy on test set',accuracy_score(LR_clf.predict(X_test),y_test))

accuracy on train set 0.9016393442622951
accuracy on test set 0.8586956521739131


In [26]:
from sklearn.svm import SVC
SVC_clf =SVC(C=1.0,kernel='poly',probability=True).fit(X_train, y_train)

from sklearn.metrics import accuracy_score
print('accuracy on train set',accuracy_score(SVC_clf.predict(X_train),y_train))
print('accuracy on test set',accuracy_score(SVC_clf.predict(X_test),y_test))

accuracy on train set 0.9180327868852459
accuracy on test set 0.8586956521739131


In [27]:
from sklearn.ensemble import RandomForestClassifier
RF_clf = RandomForestClassifier(max_features = 'sqrt',n_estimators=100).fit(X_train, y_train)

from sklearn.metrics import accuracy_score
print('accuracy on train set',accuracy_score(RF_clf.predict(X_train),y_train))
print('accuracy on test set',accuracy_score(RF_clf.predict(X_test),y_test))

accuracy on train set 1.0
accuracy on test set 0.8695652173913043


In [18]:
from sklearn.ensemble import GradientBoostingClassifier
GB_clf = GradientBoostingClassifier(n_estimators = 1000,learning_rate=0.01,subsample=0.5,max_depth=7).fit(X_train, y_train)

from sklearn.metrics import accuracy_score
print('accuracy on train set',accuracy_score(GB_clf.predict(X_train),y_train))
print('accuracy on test set',accuracy_score(GB_clf.predict(X_test),y_test))

accuracy on train set 1.0
accuracy on test set 0.8913043478260869


In [30]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(
    estimators=[\
    ('LR', LR_clf),
    ('SVC_clf', SVC_clf),
    ('RF_clf', RF_clf),
    ('GB_clf',GB_clf)], 
    voting='soft', 
    weights=[1, 1, 1, 1])

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

scores = cross_val_score(eclf,X_train,y_train,cv=10,scoring='accuracy')

In [36]:
scores.mean()

0.871921921921922

In [34]:
eclf.fit(X_train, y_train)
print('accuracy on train set',accuracy_score(eclf.predict(X_train),y_train))
print('accuracy on test set',accuracy_score(eclf.predict(X_test),y_test))

accuracy on train set 0.9863387978142076
accuracy on test set 0.9130434782608695
